In [17]:
from opendbc.car import structs
from opendbc.car.hyundai.values import CAR, HyundaiFlags
from opendbc.car.hyundai.fingerprints import FW_VERSIONS

TEST_PLATFORMS = set(CAR.with_flags(HyundaiFlags.EV | HyundaiFlags.HYBRID)) - set(CAR.with_flags(HyundaiFlags.CANFD))  # CAN hybrid and electric vehicles only
#TEST_PLATFORMS = [CAR.HYUNDAI_IONIQ_PHEV, ]

print(f"Found {len(TEST_PLATFORMS)} qualifying vehicles:")
for platform in TEST_PLATFORMS:
  print(f"  {platform}")

Found 21 qualifying vehicles:
  HYUNDAI_IONIQ_EV_LTD
  KIA_NIRO_EV
  HYUNDAI_IONIQ_HEV_2022
  HYUNDAI_IONIQ_PHEV
  HYUNDAI_KONA_HEV
  KIA_NIRO_PHEV
  HYUNDAI_IONIQ
  KIA_K5_HEV_2020
  HYUNDAI_KONA_EV_2022
  HYUNDAI_IONIQ_PHEV_2019
  HYUNDAI_SANTA_FE_PHEV_2022
  HYUNDAI_SONATA_HYBRID
  HYUNDAI_AZERA_HEV_6TH_GEN
  HYUNDAI_IONIQ_EV_2020
  KIA_NIRO_PHEV_2022
  KIA_OPTIMA_H_G4_FL
  HYUNDAI_SANTA_FE_HEV_2022
  HYUNDAI_ELANTRA_HEV_2021
  HYUNDAI_KONA_EV
  KIA_OPTIMA_H
  KIA_NIRO_HEV_2021


In [20]:
from openpilot.tools.lib.logreader import LogReader
from openpilot.tools.lib.comma_car_segments import get_comma_car_segments_database
from opendbc.car.hyundai.values import CAR

database = get_comma_car_segments_database()
TEST_SEGMENTS = []

MAX_SEGS_PER_PLATFORM = 1115  # TODO: Increase this to search more segments

print("Collecting segments from commaCarSegments dataset:")
for platform in TEST_PLATFORMS:
  assert(platform in database)
  #if platform not in database:
  #  print(f"Skipping platform: {platform}, no data available")
  #  continue
  
  all_segments = database[platform]

  #NUM_SEGMENTS = min(len(all_segments), MAX_SEGS_PER_PLATFORM)
  NUM_SEGMENTS = len(all_segments)

  print(f"Got {len(all_segments)} segments for platform {platform}, sampling {NUM_SEGMENTS} segments")

  TEST_SEGMENTS.extend(all_segments)

print(f"Collected {len(TEST_SEGMENTS)} segments for analysis")


Got 9 segments for platform HYUNDAI_IONIQ_EV_LTD, sampling 9 segments
Got 1201 segments for platform KIA_NIRO_EV, sampling 1201 segments
Got 4 segments for platform HYUNDAI_IONIQ_HEV_2022, sampling 4 segments
Got 1115 segments for platform HYUNDAI_IONIQ_PHEV, sampling 1115 segments
Got 3 segments for platform HYUNDAI_KONA_HEV, sampling 3 segments
Got 86 segments for platform KIA_NIRO_PHEV, sampling 86 segments
Got 348 segments for platform HYUNDAI_IONIQ, sampling 348 segments
Got 3 segments for platform KIA_K5_HEV_2020, sampling 3 segments
Got 440 segments for platform HYUNDAI_KONA_EV_2022, sampling 440 segments
Got 3 segments for platform HYUNDAI_IONIQ_PHEV_2019, sampling 3 segments
Got 1050 segments for platform HYUNDAI_SANTA_FE_PHEV_2022, sampling 1050 segments
Got 1062 segments for platform HYUNDAI_SONATA_HYBRID, sampling 1062 segments
Got 4 segments for platform HYUNDAI_AZERA_HEV_6TH_GEN, sampling 4 segments
Got 274 segments for platform HYUNDAI_IONIQ_EV_2020, sampling 274 segment

In [22]:
import copy
import sys
import time
import matplotlib.pyplot as plt
import numpy as np

from opendbc.can.parser import CANParser
from opendbc.car import Bus
from opendbc.car.hyundai.values import DBC

from openpilot.selfdrive.pandad import can_capnp_to_list
from openpilot.tools.lib.logreader import LogReader

analyzed_segments_count = 0
dot_counter = 0

print("Analyzing segments", end="", flush=True)
for segment in TEST_SEGMENTS:
  lr = LogReader(segment)
  CP = lr.first("carParams")
  if CP is None:
    continue

  can_msgs = [msg for msg in lr if msg.which() == "can"]
  parser_messages = [("ELECT_GEAR", 0)]
  cp = CANParser(DBC[platform][Bus.pt], parser_messages, 0)

  parsed_message_history = []

  for msg in can_msgs:
    cp.update_strings(can_capnp_to_list([msg.as_builder().to_bytes()]))
    parsed_message_history.append(copy.copy(cp.vl))

  for message in parsed_message_history:
    if message["ELECT_GEAR"]["Elect_Gear_Shifter"] in (4, 4.0):  # Sport gear
      gear = message["ELECT_GEAR"]["Elect_Gear_Shifter"]
      print(f"\nNon-Drive gear found in segment {segment:<44} for {CP.carFingerprint}")
      print(f"  {"ELECT_GEAR":<15} gear={gear}")
      break

  analyzed_segments_count += 1

  dot_counter = (dot_counter + 1) % 4
  sys.stdout.write(f"\rAnalyzing segment {analyzed_segments_count}/{len(TEST_SEGMENTS)} {'.' * dot_counter}{' ' * (3 - dot_counter)}")
  sys.stdout.flush()

print(f"\nAnalysis finished. Total segments analyzed: {analyzed_segments_count}")


Analyzing segment 1220/9107    
Non-Drive gear found in segment 201c5aaced09a24a/2023-05-20--12-39-01/15/s   for HYUNDAI IONIQ PHEV 2020
  ELECT_GEAR      gear=4.0
Analyzing segment 1224/9107    
Non-Drive gear found in segment b28e9dbe4004e915/2023-06-11--19-55-28/2/s    for HYUNDAI IONIQ PHEV 2020
  ELECT_GEAR      gear=4.0
Analyzing segment 9107/9107 ...
Analysis finished. Total segments analyzed: 9107
